# Start with a system game!

Go to http://existencia.org/intuition/thermo.html.

Don't click the "show graph" button until at least 30 minutes in.

# Basic feedback

Reproduce Roe & Baker (2017)

In [ ]:
feedback = VALUE
final.deltaTs = 1.2
additional.deltaT = 1.2
for (tt in 2:100) {
    additional.deltaT = additional.deltaT * feedback
    final.deltaTs[tt] = additional.deltaT
}

In [ ]:
plot(final.deltaTs)

In [ ]:
plot(cumsum(final.deltaTs))

In [ ]:
1.2 / (1 - VALUE)

# Constructing some simple data

In [ ]:
tt = seq(0, 10, by=.1)

In [ ]:
tt

In [ ]:
yy = AMPLITUDE * cos(FREQUENCY * (tt - PHASESHIFT)) + OFFSET

In [ ]:
plot(tt, yy)

## Connecting parameter fit with optimization

We can see that our data is a sinusoid, so it has the form $A cos(f (t - d)) + B$. But what are the values of $A$, $f$, $d$ and $B$? To find the answer we need three things:

1. Our model: $A cos(f (t - d)) + B$
2. Our data: `yy` vs. `tt` above.
3. A "loss function": something that gets minimized when the parameters are correct.

R has a couple of easy-to-use functions that can fit be used to fit parameters in an arbitrary model: `optimize` and `optim`. `optimize` just optimizes 1 parameter, while `optim` handles any number.

## Case 1: A single value

Let's just try to determine the mean of the data. What should the loss function look like?

In [ ]:
lossfunc = function(BB) {
    ???
}

In [ ]:
optimize(lossfunc, c(MIN, MAX))

In [ ]:
mean(yy)

## Case 2: all four variables

In [ ]:
model = function(AA, ff, dd, BB) {
    AA * cos(ff * (tt - dd)) + BB
}

In [ ]:
lossfunc = function(par) {
    ???
}

In [ ]:
optim(c(1, 1, 0, 0), lossfunc)

In [ ]:
par = optim(c(1, 1, 0, 0), lossfunc)$par

plot(tt, yy, col=1)
points(tt, model(par[1], par[2], par[3], par[4]), col=2, pch=16)

Let's help it out!

In [ ]:
par = optim(c(AMPLITUDE, FREQUENCY, PHASESHIFT, OFFSET), lossfunc)$par

plot(tt, yy, col=1)
points(tt, model(par[1], par[2], par[3], par[4]), col=2, pch=16)

**How far away can we go and still get the answer?**

## Why does this happen?

First, let's see if we can reproduce the problem with just 2 variables.

In [ ]:
lossfunc = function(par) {
    sum((yy - model(par[1], par[2], PHASESHIFT, OFFSET))^2)
}

In [ ]:
par = optim(c(1, 1, 0, 0), lossfunc)$par

plot(tt, yy, col=1)
points(tt, model(par[1], par[2], PHASESHIFT, OFFSET), col=2, pch=16)

In [ ]:
par

Assuming that also failed, let's sample an entire grid around our points.

In [ ]:
grid = expand.grid(AA=seq(MIN1, MAX1, length.out=100), ff=seq(MIN2, MAX2, length.out=100))

In [ ]:
grid$loss = sapply(1:nrow(grid), function(ii) lossfunc(c(grid$AA[ii], grid$ff[ii])))

In [ ]:
library(tidyverse)

In [ ]:
ggplot(grid, aes(AA, ff, fill=loss)) + geom_raster() + geom_point(data=grid[which.min(grid$loss),], col='white')

## Solution 1: Sample the whole space

## Solutions not appearing in this practicum

* Random sampling
* Genetic algorithms
* Simulated annealing

## Solution 2: Linearize with manifolds

Sinusoids are the result of feedback. That means that we should be able to write something like:
$$y_t = \alpha + \beta y_{t-1}$$

Let's try it!

In [ ]:
model = function(aa, bb) {
    ???
}

In [ ]:
lossfunc = function(par) {
    ???
}

In [ ]:
par = optim(c(1, 1, 1), lossfunc)$par

plot(tt, yy, col=1)
points(tt[1:(length(yy)-1)], model(par[1], par[2]), col=2, pch=16)

We can do even better with 2 terms-- in fact, we can find an exact solution!

In [ ]:
model = function(aa, bb, cc) {
    ???
}

In [ ]:
lossfunc = function(par) {
    ???
}

In [ ]:
par = optim(c(1, 1, 1), lossfunc)$par

plot(tt, yy, col=1)
points(tt[1:(length(yy)-2)], model(par[1], par[2], par[3]), col=2, pch=16)

In [ ]:
summary(lm(yy ~ lag(yy) + lag(yy, 2)))

**This is why you get oscillations when you try to set the thermostat!**

**Why doesn't the first feedback model above (Roe & Baker) make oscillations?**